# Intro to DataFrames

References:

- DataFrame API docs: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

Rules of thumb:
- Hit tab to auto-complete
- To see all available methods, place a dot (.) after the RDD (e.g. words.) and hit tab 
- Use `.collect()` to see the contents of the RDD

Solutions for potentially challenging exercises can be found in the end of the section. Don't peek unless you're really stuck!

In [ ]:
# like in the pyspark shell, SparkSession is already defined
spark

## 1. DataFrame methods

### 1.1 Data input

In [ ]:
df = spark.read.json("../data/people/names.json")

# other supported file formats:
# spark.read.text()
# spark.read.csv()
# spark.read.parquet()
# spark.read.orc()

# generic form: 
# spark.read.load("path/to/someFile.csv", format="csv", sep=":", inferSchema="true", header="true")

# Loading data from a JDBC source
# jdbcDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "schema.tablename") \
#     .option("user", "username") \
#     .option("password", "password") \
#     .load()

### 1.2 Data output (writing to disk)

- API docs: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter

In [ ]:
# df.write.parquet("new_data.parquet")
# df.write.mode("overwrite").parquet("new_data.parquet")


# You can read from any format and write to any format (barring formatting limitations/rules):
# df.write.csv("new_data.csv",header=True)
# df.write.json("new_data.json")
# df.write.orc("new_data.orc")
# df.write.parquet("new_data.parquet")

# generic form:
# df.write.save("fileName.parquet", format="parquet")
# df.write.mode("overwrite").save("fileName.parquet", format="parquet")

# Saving data to a JDBC source
# jdbcDF.write \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "schema.tablename") \
#     .option("user", "username") \
#     .option("password", "password") \
#     .save()


### Exploring DataFrames

In [ ]:
df.head(5)

In [ ]:
df.show(5)

In [ ]:
df.take(5)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
df.describe().show()

### Selecting specific columns in a dataframe

In [ ]:
# selecting a column
df['name']

In [ ]:
# creating a new dataframe with only selected columns
df.select('name')

In [ ]:
# creating a new dataframe with only selected columns
df.select(['name', 'age']).show()

In [ ]:
# renaming columns
df = df.withColumnRenamed('AGE', 'age')

### Filtering
`.filter()` takes in either (i) a `Column` of `types.BooleanType` or (ii) a string of SQL expression.

In [ ]:
# filter using SQL expressions
# df.where('age >= 25').show() is also possible because .where() is an alias for .filter()
df.filter('age >= 25').show()

In [ ]:
# filter using a column of boolean types
df.filter(df.age >= 25).show()

In [ ]:
# df.age >= 25 returns a Column of booleans
df.age >= 25

In [ ]:
df.filter( (df.age >= 25) & (df.age <= 30) ).show()
# you can use df.age or df['age']
# you can replace & with | for 'or' operations

### groupBy

TL;DR - `.groupBy()` allows you to group rows together based on its value in some given column(s)
- `df.groupBy([cols])`
- [GroupedData operations](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) (alternatively, you can instantiate a variable with the type of GroupedData, let jupyter notebook's intellisense show you what methods are available:
    - `grouped = df.groupBy('gender')`
    - `grouped.` (and hit tab)

In [ ]:
df.head()

In [ ]:
df.groupBy('gender')

In [ ]:
grouped = df.groupBy('gender')

In [ ]:
df.groupBy('gender').count().show()

In [ ]:
df.groupBy('gender').avg().show()

In [ ]:
# calculate total age and height
df.groupBy().sum().show()

In [ ]:
# TODO: calculate average height of all people (i.e. don't groupby anything)

In [ ]:
# TODO: calculate max height for each gender

In [ ]:
# TODO: calculate min height for each gender

### 1.2 Crimes Data

In [ ]:
crimes = spark.read.csv("../data/crimes/Crimes_-_One_year_prior_to_present.csv", header=True, inferSchema=True)
# try the above without the header and inferSchema option. see what happens!

In [ ]:
crimes.printSchema()

In [ ]:
# TODO: Explore the dataset
# - what are the columns?
# - what's the schema (e.g. data type of each column)?
# - what does the first row look like?
# - how many entries are there?

In [ ]:
crimes.columns

In [ ]:
# TODO: Explore the dataset
crimes.show(5)

In [ ]:
# TODO: What columns are in the dataframe? (you can optionally rename the improperly formatted column names)

In [ ]:
# TODO: How many crimes were reported in total?

In [ ]:
# TODO: How many cases resulted in arrest, and how many didn’t?

# (hint: ...TSERRA yBpuorg dluoc uoy)

In [ ]:
# TODO: List top 10 WARDs with the highest number of cases

In [ ]:
# TODO: List top 15 categories (PRIMARY DESCRIPTION) of cases

In [ ]:
# TODO: List top 20 locations (LOCATION DESCRIPTION) where cases occur

In [ ]:
# TODO: Save one of the results to disk (choose any format)
# Note: if your dataframe ends up being partitioned, you can call `your_df.coalesce(1)` before saving (`df.coalesce(1).write...`)

In [ ]:
# TODO: submit the preceeding task as a spark job
# 1. Create a python file named jobs/top_20_crime_locations.py
# 2. define spark session object
#   - from pyspark.sql import SparkSession
#   - spark = SparkSession.builder.appName("MyAppName").getOrCreate()
# 3. Copy the code in the preceeding cell into the file 
# 4. submit the job: ${SPARK_HOME}/bin/spark-submit --master local ./jobs/top_20_crime_locations.py

In [ ]:
# TODO: Use your creativity - create any interesting DataFrames or insights into the crimes data!

### Spark SQL functions

In [ ]:
# desc(), explode(), split(), size(), max(), col()

#### Built-in aggregate functions

In [ ]:
from pyspark.sql.functions import avg

In [ ]:
df.select(avg('age')).show()

# this produces the same output:
# df.select('age').groupBy().avg().show()

In [ ]:
df.select(avg('age'), avg('height')).show()

# this produces the same output:
# df.groupBy().avg().show()

In [ ]:
from pyspark.sql.functions import countDistinct

In [ ]:
df.select(countDistinct('name')).show()
# there are 6 unique (distinct) names in the dataset

## Using SQL with Spark DataFrames

In [ ]:
df = spark.read.json("../data/people/names.json")

In [ ]:
df.createOrReplaceTempView('names')

In [ ]:
spark.sql("SELECT * FROM names")
# add .show() to see the resulting dataframe. Example:
# df = spark.sql("SELECT * FROM names")
# df.show()

In [ ]:
spark.sql("SELECT * FROM names WHERE height > 170").show()